In [ ]:
import numpy as np
import string 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
input_text=[]
label_arr = []

In [ ]:
data = pd.read_csv('fake_or_real_news.csv')

In [ ]:
data.columns

In [ ]:
text = data.iloc[:,2]
text

In [ ]:
label = data.iloc[:,3]
label

In [ ]:
for t in text:
    t = t.replace('\n',"")
    t = t.lower()
    new_t = t.translate(str.maketrans('','',string.punctuation))
    input_text.append(new_t)
    
for l in label:
    label_arr.append(l)

In [ ]:
input_text[1]

In [ ]:
label_arr

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input_text,label_arr,test_size=0.33)

In [ ]:
x_train[0],y_train[0]

In [ ]:
len(x_train) , len(x_test)

In [ ]:
idx = 1
word2idx = {'<unk>':0}

In [ ]:
for text in x_train:
    tokens = text.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx+=1


In [ ]:
len(word2idx)

In [ ]:
train_text_int,test_text_int = [],[]

In [ ]:
for text in x_train:
    tokens = text.split()
    for token in tokens:
        lines_as_int = [word2idx[token] for token in tokens]
        train_text_int.append(lines_as_int)

for text in x_test:
    tokens = text.split()
    line_as_int = [word2idx.get(token,0) for token in tokens]
    test_text_int.append(line_as_int)     

In [ ]:
train_text_int[0]
test_text_int[0]

In [ ]:
V = len(word2idx)
A0,A1 = np.ones((V,V)),np.ones((V,V))
pi0,pi1 = np.ones(V),np.ones(V)

In [ ]:
def compute_counts(text_as_int , A , pi):
    for tokens in text_as_int:
        last_index = None
        for idx in tokens:
            if last_index is None:
                pi[idx]+=1
            else:
                A[last_index,idx]+=1
            last_index = idx

In [ ]:
compute_counts([t for t,y in zip(train_text_int,y_train) if y==0 ],A0,pi0)
compute_counts([t for t,y in zip(train_text_int,y_train) if y==1 ],A1,pi1)

In [ ]:
A0/= A0.sum(axis = 1,keepdims=True)
pi0/= pi0.sum()
A1/= A1.sum(axis = 1,keepdims=True)
pi1/= pi1.sum()

In [ ]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)
logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
count0 = sum(y==0 for y in y_train)
count1 = sum(y==1 for y in y_train)
total = len(y_train)
p0 = count0/total
p1 = count1/total
logp0 = np.log(p0)
logp1 = np.log(p1)

In [ ]:
class Classifier:
    def __init__(self,logAs,logpis,logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) #number of classes
    
    def compute_log_likelihood(self,input_,class_):
        logA = self.logAs[class_]
        logpi = self.logpi[class_]
        last_index = None
        logprob = 0
        for idx in input_:
            if last_index is None:
                logprob+=logpi[idx]
            else:
                logprob+=logA[last_index,idx]
            last_index = idx
            
        return logprob
    
    
    def predict(self,inputs):
        predictions = np.zeros(len(inputs))
        for i input_ in enumerate (inputs):
            posteriors = [self._compute_log_likelihood(input_,c) + self.logperiors[c] for c in range (self.K)]
            pred = np.argmax(posteriors)
            
            prediction[i] = pred
        
        return prediction
                               